### 1) Data Preparation

In [1]:
import pandas as pd
path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/ML based speckle POC/"
df = pd.read_csv(path+"SNR_R4_ww49.1_Speckle.csv.gz")

In [ ]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [2]:
df["DELTA"].value_counts()

0    13390
1     6317
2      129
Name: DELTA, dtype: int64

In [3]:
df["SPECKLE"].value_counts()

0    13390
1     6446
Name: SPECKLE, dtype: int64

In [ ]:
df.head()

In [ ]:
pd.options.display.max_seq_items = 2000
df.columns

In [ ]:
df.shape

In [ ]:
df.describe()

### Duplicate handling

In [4]:
def duplicates(df,keep):
    """
    Check for duplicates in VID and remove duplicates if there is any
    df[DataFrame]: input df
    keep: {‘first’, ‘last’, False}: Determines which duplicates (if any) to keep. 
    - first : Drop duplicates except for the first occurrence. 
    - last : Drop duplicates except for the last occurrence. 
    - False : Drop all duplicates.
    """
    if df['VID'].duplicated().any():
        print("There are duplicates in VID")
        df_dup = df[df.duplicated(subset=['VID'])] #get the duplicates in df 
        display(df_dup)
        df = df.drop_duplicates(subset=['VID'],keep=keep) #remove duplicates from df
        return df
    else:
        print("There are no duplicates in VID")
    

In [5]:
duplicates(df,keep="first")

There are no duplicates in VID


### 2) NA handling

In [6]:
df.shape

(19836, 1615)

In [7]:
# check column for nulls
def check_colna(df):
    """
    Check each column for nulls. Returns Feature, total null and % of null for each column
    df[DataFrame]: dataframe
    
    """
    colna_df = pd.DataFrame(columns =["Feature", "Total Null", "% of Null"])
    for col in df.columns: 
        #checking if there is any null in the column

        if df[col].isnull().sum()>0: 
            
            # if null present, total number of null in the column stores here
            total_null = df[col].isnull().sum() 
            new_row = {'Feature':col, 'Total Null':total_null, '% of Null':round(total_null*100/len(df),2)}
            #append row to the dataframe
            colna_df = colna_df.append(new_row, ignore_index=True)
            
    colna_df= colna_df.sort_values("% of Null", ascending=False)    
    return colna_df  

# check rows for nulls
def check_rowna(df,supporting_fs):  
    """
    Check each row for nulls.Returns VID, total null and % of null for each row
    df[DataFrame]: dataframe
    supporting_fs[list]: all features not used for ML except VID and SPECKLE(target)
    """
    df = df.drop(supporting_fs,axis=1)    
    colrow_df = pd.DataFrame(columns =["SPECKLE","VID", "Total Null", "% of Null"])
    for i in df.index: 
        #checking if there is any null in the row
        if df.iloc[i].isnull().sum()>0:             
            # if null present, total number of null in the row stores here
            total_null = df.iloc[i].isnull().sum() 
            new_row = {'SPECKLE':df.iloc[i,0],'VID':df.iloc[i,1], 'Total Null':total_null, '% of Null':round(total_null*100/(len(df.columns)-2),2)}
            #append row to the dataframe
            colrow_df = colrow_df.append(new_row, ignore_index=True)
            
    colrow_df= colrow_df.sort_values("% of Null", ascending=False)    
    return colrow_df   

# Drop columns based on NA threshold limit
def drop_NAcol(df,NA_limit):
    '''
    Drops columns based on proportion of NA in column
    df[DataFrame]: df
    NA_limit[float/int]: Columns with proportion of NA above NA_limit will be dropped
    '''
    threshold = len(df)*(1-NA_limit)
    df=df.dropna(axis=1, thresh=threshold)
    print(df.shape)
    return df


#### a) Check column for nulls

In [8]:
#check column na for whole df
colna_df =check_colna(df)
colna_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,16129,81.31
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,13847,69.81
399,IDV_0134_PSVT3GVTO12_FULLDIE_TALL_0950_MED@132110,13847,69.81
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,13847,69.81
393,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0950_ME...,13847,69.81
...,...,...,...
1472,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1473,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1407,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1406,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85


In [ ]:
# colna_df.to_csv(path+"/DataPreparation/NA_Cols.csv",index=False)

In [9]:
#check column na for speckle
colna_speckle_df =check_colna(df.loc[df['SPECKLE'] == 1])
colna_speckle_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,5268,81.73
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,4845,75.16
399,IDV_0134_PSVT3GVTO12_FULLDIE_TALL_0950_MED@132110,4845,75.16
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,4845,75.16
393,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0950_ME...,4845,75.16
...,...,...,...
1472,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1473,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1406,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1405,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56


In [10]:
#check column na for non-speckle
colna_nonspeckle_df =check_colna(df.loc[df['SPECKLE'] == 0])
colna_nonspeckle_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,10861,81.11
875,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
882,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
889,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
888,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
...,...,...,...
820,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
822,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
824,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
826,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59


#### b) Check row for nulls

In [ ]:
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
rowna_df = check_rowna(df,supporting_fs)
rowna_df

In [ ]:
rowna_df.loc[rowna_df["% of Null"] ==100.00]

In [ ]:
# rowna_df.to_csv(path+"/DataPreparation/NA_Rows_aftercolremoval.csv",index=False)

#### c) Drop columns based on threshold limit

In [ ]:
df = drop_NAcol(df,0.8) #drop columns with >80% NA 

In [ ]:
check_colna(df)

#### d) Row NA handling

In [11]:
#check rows for NA after removing columns
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
rowna_df = check_rowna(df,supporting_fs)
rowna_df

,SPECKLE,VID,Total Null,% of Null
8213,0,M1GC346401871,1593,100.00
10375,0,M11H9B4300835,1593,100.00
10377,0,M11H9B4301228,1593,100.00
10378,0,M11H9B4300032,1593,100.00
10379,0,M11H9B4301227,1593,100.00
...,...,...,...,...
11806,0,M05C0K5600161,1,0.06
11811,0,M0U25B1100396,1,0.06
11814,0,M0U25B1100907,1,0.06
11815,0,M0U25B1100798,1,0.06


In [12]:
#get rows with 100% NA
rowallNA_df = rowna_df.loc[rowna_df["% of Null"] ==100.00]
rowallNA_df

,SPECKLE,VID,Total Null,% of Null
8213,0,M1GC346401871,1593,100.0
10375,0,M11H9B4300835,1593,100.0
10377,0,M11H9B4301228,1593,100.0
10378,0,M11H9B4300032,1593,100.0
10379,0,M11H9B4301227,1593,100.0
...,...,...,...,...
5463,0,M10P67G000951,1593,100.0
5464,0,M10P67G001238,1593,100.0
5468,0,M17U1C0300567,1593,100.0
5465,0,M10P67G000323,1593,100.0


In [13]:
#drop rows with 100% NA
df = df[~df["VID"].isin(rowallNA_df["VID"])]
df = df.reset_index(drop=True)
df

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,M10P67G000001,N0286920_293_6_-3,N0286920,N028692,293,6,-3,H0280460_443_2_1,1,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,M10P67G000962,N0286920_225_5_-5,N0286920,N028692,225,5,-5,H0280460_443_2_18,1,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,M10P67G000808,N0292860_135_6_0,N0292860,N029286,135,6,0,H0256240_667_-10_6,1,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,M17U1C0300573,N0293360_159_3_1,N0293360,N029336,159,3,1,H0280460_443_0_-14,1,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,M10P67G000433,N0293340_045_1_5,N0293340,N029334,45,1,5,H0280460_444_9_13,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,0,M1G88H4701199,N0326460_212_4_6,N0326460,N032646,212,4,6,H0310990_073_5_-2,1,...,-7.190000e-07,-7.620000e-07,-7.720000e-07,-1.810000e-07,-1.860000e-07,-1.720000e-07,-1.780000e-07,-7.080000e-07,-7.430000e-07,-1.720000e-07
6373,0,M1G88H4700911,N0326460_209_-1_-2,N0326460,N032646,209,-1,-2,H0310990_073_-3_18,1,...,-9.140000e-07,-9.750000e-07,-9.730000e-07,-2.420000e-07,-2.400000e-07,-2.330000e-07,-2.310000e-07,-8.940000e-07,-8.840000e-07,-2.310000e-07
6374,0,M1G88H4700361,N0370830_775_-3_0,N0370830,N037083,775,-3,0,H0310990_083_-14_-3,1,...,-1.010000e-06,-9.770000e-07,-1.000000e-06,-2.500000e-07,-2.510000e-07,-3.010000e-07,-2.310000e-07,-9.300000e-07,-9.430000e-07,-2.310000e-07
6375,0,M1G88H4700053,N0342690_197_1_-6,N0342690,N034269,197,1,-6,H0310990_073_-10_15,14,...,-1.060000e-06,-1.120000e-06,-1.130000e-06,-2.610000e-07,-2.700000e-07,-2.700000e-07,-2.720000e-07,-1.090000e-06,-1.110000e-06,-2.270000e-07


In [14]:
df["SPECKLE"].value_counts()

0    4608
1    1769
Name: SPECKLE, dtype: int64

In [15]:
#check row na
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
check_rowna(df,supporting_fs)

,SPECKLE,VID,Total Null,% of Null
573,1,M1888SS100613,1310,82.23
579,1,M1888SS100734,1310,82.23
581,1,M1888SS100614,1310,82.23
582,1,M1888SS100616,1310,82.23
577,1,M1888SS100692,1310,82.23
...,...,...,...,...
1536,0,M0LN682800123,1,0.06
1535,0,M0LN682801353,1,0.06
1534,0,M0LN682801352,1,0.06
1533,0,M0LN682800122,1,0.06


In [16]:
df = df.drop(['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110'],axis=1)

In [17]:
df.shape

(6377, 1614)

In [18]:
#NA imputation
def NA_impute(df,imptype):
    """
    Impute NA
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.fillna(df.mean())
    if imptype == "median":
        df = df.fillna(df.median())
    return df

In [19]:
df = NA_impute(df,imptype="median")
df

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,M10P67G000001,N0286920_293_6_-3,N0286920,N028692,293,6,-3,H0280460_443_2_1,1,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,M10P67G000962,N0286920_225_5_-5,N0286920,N028692,225,5,-5,H0280460_443_2_18,1,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,M10P67G000808,N0292860_135_6_0,N0292860,N029286,135,6,0,H0256240_667_-10_6,1,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,M17U1C0300573,N0293360_159_3_1,N0293360,N029336,159,3,1,H0280460_443_0_-14,1,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,M10P67G000433,N0293340_045_1_5,N0293340,N029334,45,1,5,H0280460_444_9_13,1,...,-8.520000e-07,-8.570000e-07,-8.640000e-07,-2.150000e-07,-2.250000e-07,-2.190000e-07,-2.170000e-07,-8.545000e-07,-8.510000e-07,-2.130000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,0,M1G88H4701199,N0326460_212_4_6,N0326460,N032646,212,4,6,H0310990_073_5_-2,1,...,-7.190000e-07,-7.620000e-07,-7.720000e-07,-1.810000e-07,-1.860000e-07,-1.720000e-07,-1.780000e-07,-7.080000e-07,-7.430000e-07,-1.720000e-07
6373,0,M1G88H4700911,N0326460_209_-1_-2,N0326460,N032646,209,-1,-2,H0310990_073_-3_18,1,...,-9.140000e-07,-9.750000e-07,-9.730000e-07,-2.420000e-07,-2.400000e-07,-2.330000e-07,-2.310000e-07,-8.940000e-07,-8.840000e-07,-2.310000e-07
6374,0,M1G88H4700361,N0370830_775_-3_0,N0370830,N037083,775,-3,0,H0310990_083_-14_-3,1,...,-1.010000e-06,-9.770000e-07,-1.000000e-06,-2.500000e-07,-2.510000e-07,-3.010000e-07,-2.310000e-07,-9.300000e-07,-9.430000e-07,-2.310000e-07
6375,0,M1G88H4700053,N0342690_197_1_-6,N0342690,N034269,197,1,-6,H0310990_073_-10_15,14,...,-1.060000e-06,-1.120000e-06,-1.130000e-06,-2.610000e-07,-2.700000e-07,-2.700000e-07,-2.720000e-07,-1.090000e-06,-1.110000e-06,-2.270000e-07


In [20]:
check_rowna(df,supporting_fs)

,SPECKLE,VID,Total Null,% of Null


### 3) Drop supporting features

In [23]:
supporting_fs = ['VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
df_sf = df[supporting_fs] #keep supporting fs
df = df.drop(supporting_fs,axis=1)
df

,SPECKLE,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,6016.90,12304.2,5791.55,12112.7,5994.37,12293.0,5887.32,12214.1,5245.07,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,6073.24,12214.1,6005.63,12163.4,6078.87,12174.7,6061.97,12236.6,5295.77,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,6343.66,12231.0,6163.38,12028.2,6264.79,12152.1,6219.72,12129.6,5695.77,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,5566.20,11453.5,5442.25,11357.8,5554.93,11414.1,5476.06,11391.5,4811.27,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,6247.89,12293.0,6112.68,12180.3,6219.72,12270.4,6163.38,12259.2,5470.42,...,-8.520000e-07,-8.570000e-07,-8.640000e-07,-2.150000e-07,-2.250000e-07,-2.190000e-07,-2.170000e-07,-8.545000e-07,-8.510000e-07,-2.130000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,0,6664.79,12450.7,6602.82,12371.8,6670.42,12462.0,6636.62,12416.9,5954.93,...,-7.190000e-07,-7.620000e-07,-7.720000e-07,-1.810000e-07,-1.860000e-07,-1.720000e-07,-1.780000e-07,-7.080000e-07,-7.430000e-07,-1.720000e-07
6373,0,6208.45,12056.3,6095.77,12000.0,6163.38,12056.3,6185.92,12078.9,5453.52,...,-9.140000e-07,-9.750000e-07,-9.730000e-07,-2.420000e-07,-2.400000e-07,-2.330000e-07,-2.310000e-07,-8.940000e-07,-8.840000e-07,-2.310000e-07
6374,0,6202.82,12169.0,6163.38,12118.3,6253.52,12185.9,6264.79,12247.9,5526.76,...,-1.010000e-06,-9.770000e-07,-1.000000e-06,-2.500000e-07,-2.510000e-07,-3.010000e-07,-2.310000e-07,-9.300000e-07,-9.430000e-07,-2.310000e-07
6375,0,5526.76,11549.3,5397.18,11493.0,5464.79,11509.9,5459.15,11554.9,4867.61,...,-1.060000e-06,-1.120000e-06,-1.130000e-06,-2.610000e-07,-2.700000e-07,-2.700000e-07,-2.720000e-07,-1.090000e-06,-1.110000e-06,-2.270000e-07


### 5) Handling unary columns

In [24]:
def unary(df):
    """
    Checks for unary columns. If there are unary columns, the unary columns will be printed and removed from the df
    df[DataFrame]: input dataframe
    """
    unarycolumns = [col for col in df.columns if len(df[col].unique())==1]
    if unarycolumns:
        print("The unary column are:",unarycolumns)
        df = df.drop(unarycolumns,axis=1)
        print("Unary columns dropped!")
        return df 
    else:
        print("There are no unary columns!")

In [25]:
unary(df)

There are no unary columns!


In [ ]:
# supporting_fs = ['SPECKLE','VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# sf_df = df[supporting_fs]
# df1 = df.drop(supporting_fs,axis=1) 
# df1.head()

In [ ]:
# df1.columns.difference(df2.columns)

In [ ]:
# df1['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110']

In [ ]:
# df2 = df1.drop(['PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110'],axis=1)
# df_neg = df2.loc[:,((df2 == -555).any()) | ((df2 == -999).any())]

In [ ]:
# df_neg_sf = pd.concat([sf_df,df_neg],axis=1)
# df_neg_sf

In [ ]:
# df_neg_sf.to_csv(path+"/DataPreparation/Negative_Cols_555_999.csv",index=False)

### 6) Train-validation split

In [26]:
from sklearn.model_selection import train_test_split
from collections import Counter
def randomsamp(df,val_size):
    """
    Split whole dataset into train and validation using random sampling
    Returns X_train, X_val, y_train, y_val
    df[DataFrame]: input dataframe    
    val_size[float]:Should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
                    Train size is complement of val size
    """
    X= df.drop(["SPECKLE"],axis=1)
    y= df["SPECKLE"] 
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = val_size,random_state=42,stratify= df["SPECKLE"])
    if type(val_size)==float:
        print("Train-val split completed with",(1-val_size)*100,"-",val_size*100,"split in train-val")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_val is:",X_val.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_val is:",y_val.shape)
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)        
           
    return X_train, X_val, y_train, y_val

def targetrandomsamp(df,speckle_test_size,nonspeckle_test_size):
    """
    Dataset is split into speckle/non-speckle first. The speckle/non-speckle datasets are then split 
    into train and validation using random sampling, followed by merging of the speckle/non-speckle  
    to return X_train, X_val, y_train, y_val
    
    df[DataFrame]: input dataframe  
    
    speckle_test_size[float/int]: Val size for speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    nonspeckle_test_size[float/int]: Val size for non-speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    """
    #split data into speckle/nonspeckle
    df_s = df.loc[df["SPECKLE"] == 1] #speckle
    df_ns = df.loc[df["SPECKLE"] == 0] #nonspeckle
    
    #split speckle/nonspeckle data into train/validation
    print("For speckle data:")
    X_train_s, X_val_s, y_train_s, y_val_s = randomsamp(df_s,val_size=speckle_test_size)
    print("\nFor non-speckle data:")
    X_train_ns, X_val_ns, y_train_ns, y_val_ns = randomsamp(df_ns,val_size=nonspeckle_test_size)
    
    #concat the speckle/non-speckle train and validation
    X_train = pd.concat([X_train_s,X_train_ns], ignore_index=True)
    X_val = pd.concat([X_val_s,X_val_ns], ignore_index=True)
    y_train = pd.concat([y_train_s,y_train_ns], ignore_index=True)
    y_val = pd.concat([y_val_s,y_val_ns], ignore_index=True)
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)  
    
    print("\nFinal dataset:")
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    return X_train, X_val, y_train, y_val

In [27]:
df["SPECKLE"].value_counts()

0    4608
1    1769
Name: SPECKLE, dtype: int64

In [30]:
#train-test split on whole data
X_train, X_val, y_train, y_val = randomsamp(df,val_size=0.3)

Train-val split completed with 70.0 - 30.0 split in train-val
Shape of X_train is: (4463, 1592)
Shape of X_val is: (1914, 1592)
Shape of y_train is: (4463,)
Shape of y_val is: (1914,)
Distribution of y_train: Counter({0: 3225, 1: 1238})
Distribution of y_val: Counter({0: 1383, 1: 531})


In [29]:
#train-test split on speckle/non-speckle then merge
X_train, X_val, y_train, y_val = targetrandomsamp(df,speckle_test_size=500,nonspeckle_test_size=2000)

For speckle data:
Shape of X_train is: (1269, 1592)
Shape of X_val is: (500, 1592)
Shape of y_train is: (1269,)
Shape of y_val is: (500,)
Distribution of y_train: Counter({1: 1269})
Distribution of y_val: Counter({1: 500})

For non-speckle data:
Shape of X_train is: (2608, 1592)
Shape of X_val is: (2000, 1592)
Shape of y_train is: (2608,)
Shape of y_val is: (2000,)
Distribution of y_train: Counter({0: 2608})
Distribution of y_val: Counter({0: 2000})

Final dataset:
Distribution of y_train: Counter({0: 2608, 1: 1269})
Distribution of y_val: Counter({0: 2000, 1: 500})


### 7) Scaling

In [31]:
#To scale X train and X validation (test) data using various scalers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def scale_data(X_train,X_val,scaler_type):
    """
    Scaling X train and validation with standardization or normalization
    params:
    X_train[DataFrame]: input X train
    X_val[DataFrame]: input X validation (test)
    scaler_type[None/string]: input scaling method
    - "Standardization" for Standard Scaler
    - "Normalization" for Min Max Scaler    

    """           
    if scaler_type == "Standardization":
        scaler = StandardScaler()
    if scaler_type == "Normalization":
        scaler = MinMaxScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled,columns= X_train.columns)
    X_val_scaled = scaler.transform(X_val)
    X_val_scaled = pd.DataFrame(X_val_scaled,columns= X_val.columns)
       
    return X_train_scaled, X_val_scaled

In [37]:
#X train-test scaled using Standard Scaler
X_train_scaled, X_val_scaled = scale_data(X_train,X_val,scaler_type = "Standardization")

In [34]:
#X train-test scaled using Standard Scaler
X_train_scaled, X_val_scaled = scale_data(X_train,X_val,scaler_type = "Normalization")

In [38]:
X_train_scaled

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1.508414,1.604347,1.650496,1.643010,1.463659,1.467708,1.687350,1.714588,1.606910,1.731650,...,0.024400,0.105669,0.320966,-0.777714,0.253389,0.305936,0.449498,-0.151078,-0.315189,-0.715357
1,1.087282,1.064858,1.227366,1.216943,1.133399,1.011858,1.230072,1.167966,1.222037,1.171273,...,0.690464,0.593801,0.716449,0.960975,0.335471,0.699969,-0.273456,0.631304,0.718159,0.679346
2,0.718787,0.226205,0.416374,-0.386120,0.646755,0.277924,0.368306,-0.512510,0.757014,0.110217,...,1.217015,1.423624,1.164664,0.908287,1.254795,0.682837,1.384047,1.431880,1.468023,1.491452
3,0.841629,1.364378,0.768967,1.399543,0.907477,1.507485,0.684885,1.248827,0.580630,1.451639,...,-0.232125,-0.107334,-0.043757,-0.127901,-0.173440,-0.139491,-0.132392,-0.242053,-0.077428,0.043785
4,0.087074,0.985105,0.046131,0.973476,0.125311,0.853103,0.034151,0.985758,-0.173034,0.810955,...,-1.694766,-1.593916,-1.638874,-1.304590,-1.486760,-1.544301,-1.719363,-1.884145,-1.874356,-1.898207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4458,-0.123476,0.246054,-0.253606,0.100813,-0.065873,0.357125,-0.352793,-0.046750,-0.253185,0.190169,...,-0.659666,-0.497839,-0.760022,-0.602089,-0.288356,0.117486,-0.467419,-0.255699,-0.264893,0.308602
4459,0.455568,0.805393,0.486880,0.790876,0.525078,0.912592,0.421064,0.783426,0.275965,0.590642,...,0.235920,0.163357,0.215504,0.381412,-0.058525,-0.053832,0.114470,0.271954,0.132900,0.079094
4460,0.771425,0.166301,0.698461,0.141511,0.664124,0.079744,0.544187,0.013986,0.740989,0.110217,...,1.482541,1.547876,1.529387,1.540538,1.484626,1.436637,1.172451,1.486465,1.408582,1.279599
4461,1.561051,1.684101,1.527103,1.440241,1.654843,1.884132,1.458711,1.451519,1.735192,1.711751,...,1.388032,1.188434,1.085567,1.505413,1.566708,1.299582,1.489845,1.359101,1.303418,1.350217


In [39]:
X_val_scaled

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,-2.229167,-1.570911,-2.157708,-1.562756,-2.099418,-1.625731,-2.076357,-1.565503,-2.530202,-2.091847,...,-0.614662,-0.795155,-0.760022,-0.514277,-0.567436,-1.133137,-0.696648,-0.792450,-0.639825,-0.750666
1,-2.018586,-1.930334,-1.963777,-1.907787,-2.029910,-1.962954,-1.988417,-1.848697,-1.792591,-1.931943,...,-0.322134,-0.364712,-0.584251,-0.672339,-0.222690,-0.019568,-0.802446,-0.519526,-0.338051,0.096749
2,1.771633,2.143482,1.844458,2.069076,1.672212,1.963686,1.722533,1.957531,1.671037,2.031916,...,1.082002,0.868929,0.905403,1.241975,0.992131,0.939814,0.731626,1.054336,1.074802,0.908855
3,-1.035944,-1.351499,-0.994092,-1.400685,-1.039192,-1.407839,-1.003527,-1.444032,-0.974800,-1.371211,...,0.127910,0.167795,0.074888,0.276036,-0.091358,-0.242282,-0.114759,-0.105591,0.000302,0.026131
4,1.052180,0.545928,1.121591,0.709480,1.081261,0.714412,1.036616,0.641469,1.478629,1.031268,...,0.564452,0.527237,0.558256,0.469224,0.532469,0.477255,0.766892,0.526683,0.324938,0.820582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909,0.192349,0.426120,0.345836,0.547409,0.142711,0.436679,0.280366,0.459262,0.340120,0.470536,...,0.235920,0.158920,0.057310,0.451662,0.138473,0.186014,0.343700,0.176430,0.114611,0.079094
1910,-0.649915,-1.111530,-0.747242,-1.501889,-0.865377,-1.348349,-0.792464,-1.524893,-0.285262,-1.171152,...,1.667059,1.712066,1.648032,1.540538,1.796539,1.642219,1.525112,1.695707,1.692067,1.509107
1911,2.017285,1.544444,2.144164,1.724045,2.054610,1.527197,2.056688,1.633726,2.280395,1.911809,...,0.811976,0.864492,0.861460,0.925850,1.074213,1.008341,1.101919,0.704083,0.809606,1.385526
1912,-0.842930,-0.652503,-0.747242,-0.426458,-0.969684,-0.733393,-0.651766,-0.370913,-0.910645,-0.630674,...,-1.154714,-1.238911,-1.199448,-1.497778,-1.174847,-1.030346,-1.260905,-1.201836,-1.142782,-0.874247


### 2) Sampling to address class imbalance

In [40]:
Counter(y_train)

Counter({0: 3225, 1: 1238})

In [42]:
from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

#SMOTE
def SMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose SMOTE or SMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    over_amt [int] : amount of synthetic data to be generated with SMOTE (speckle)
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #SMOTE only
        sm = SMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_s, y_s = sm.fit_resample(X_train_scaled, y_train)
        
    else: #SMOTE + random undersampling
        sm = SMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', sm), ('u', under)])
        X_s, y_s = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_s))     
    print("Ratio of class distribution after sampling :",round(Counter(y_s)[0]/Counter(y_s)[1],2)) 
    
    return X_s, y_s

#BorderlineSMOTE
def BorderlineSMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose BorderlineSMOTE or BorderlineSMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with BorderlineSMOTE
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #BorderlineSMOTE only
        bsm = BorderlineSMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_bs, y_bs = bsm.fit_resample(X_train_scaled, y_train)
        
    else: #BorderlineSMOTE + random undersampling
        bsm = BorderlineSMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', bsm), ('u', under)])
        X_bs, y_bs = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_bs))     
    print("Ratio of class distribution after sampling :",round(Counter(y_bs)[0]/Counter(y_bs)[1],2)) 
    
    return X_bs, y_bs

#ADASYN
def ADASYN_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose ADASYN or ADASYN + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with ADASYN
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #ADASYN only
        ad = ADASYN(sampling_strategy= {1:over_amt},random_state=42)
        X_a, y_a = ad.fit_resample(X_train_scaled, y_train)
        
    else: #ADASYN + random undersampling
        ad = ADASYN(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', ad), ('u', under)])
        X_a, y_a = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_a))     
    print("Ratio of class distribution after sampling :",round(Counter(y_a)[0]/Counter(y_a)[1],2)) 
    
    return X_a, y_a

In [44]:
#SMOTE
X_s, y_s = SMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

Class distribution before sampling: Counter({0: 3225, 1: 1238})
Ratio of class distribution before sampling : 2.61
Class distribution after sampling: Counter({0: 2000, 1: 2000})
Ratio of class distribution after sampling : 1.0


In [45]:
#BorderlineSMOTE
X_bs, y_bs = SMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

Class distribution before sampling: Counter({0: 3225, 1: 1238})
Ratio of class distribution before sampling : 2.61
Class distribution after sampling: Counter({0: 2000, 1: 2000})
Ratio of class distribution after sampling : 1.0


In [46]:
#ADASYN
X_a, y_a = SMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=None)      

Class distribution before sampling: Counter({0: 3225, 1: 1238})
Ratio of class distribution before sampling : 2.61
Class distribution after sampling: Counter({0: 3225, 1: 2000})
Ratio of class distribution after sampling : 1.61


In [ ]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [ ]:
# df.to_csv(path+"SNR_R2_ww35.2_Speckle.csv",index=False)

In [ ]:
# supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# df1 = df.drop(supporting_fs,axis=1) 
# df1.head()

In [ ]:
import spacy
spacy.cli.download("en_core_web_sm")

### Feature Selection

### Model Building